# work-flow of training in pytorch

### importing the necessary libraries

In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

### Loading the dataset

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
df.shape

(569, 33)

In [4]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

### Splitting the data

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

### Scaling the data

In [8]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
y_train

,diagnosis
367,B
484,B
182,M
38,M
7,M
...,...
191,B
244,M
194,M
451,M


### Label encoding

In [11]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

### Converting numpy arrays to pytorch tensors

In [13]:
x_train_tensor = torch.from_numpy(X_train)
x_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [14]:
x_train_tensor.shape

torch.Size([455, 30])

### Building the model from scratch

In [30]:
class MySimpleNN():
    def __init__(self, x):
        self.weights = torch.rand(x.shape[1], 1, dtype = torch.float64, requires_grad=True) 
        self.bias = torch.zeros(1, dtype = torch.float64, requires_grad=True) 
        
    def forward(self,x):
        z = torch.matmul(x,self.weights) + self.bias
        y_pred = torch.sigmoid(z)
        return y_pred
    
    def loss_function(self,y_pred,y):
        # clamp predictions to avoid log(0)
        epsilon = 1e-7
        y_pred = torch.clamp(y_pred, epsilon, 1-epsilon)
        
        loss = -(y_train_tensor * torch.log(y_pred) + (1 - y_train_tensor) * torch.log(1 - y_pred)).mean()
        return loss

### Important parameters

In [31]:
learning_rate = 0.1
epochs = 25

### Creating model

In [32]:
model = MySimpleNN(x_train_tensor)
print("Model Weights:\n",model.weights)
print("Model bias:\n",model.bias)

Model Weights:
 tensor([[0.9599],
        [0.2298],
        [0.3554],
        [0.2898],
        [0.1326],
        [0.9461],
        [0.2000],
        [0.2706],
        [0.3792],
        [0.6578],
        [0.7622],
        [0.0459],
        [0.9488],
        [0.3934],
        [0.1708],
        [0.9401],
        [0.3605],
        [0.4869],
        [0.1569],
        [0.8474],
        [0.1760],
        [0.1817],
        [0.7320],
        [0.3321],
        [0.8943],
        [0.0464],
        [0.7725],
        [0.9975],
        [0.7709],
        [0.1986]], dtype=torch.float64, requires_grad=True)
Model bias:
 tensor([0.], dtype=torch.float64, requires_grad=True)


In [33]:
# define loop
for epoch in range(epochs):

    # forward pass
    y_pred = model.forward(x_train_tensor)

    # loss calculate
    loss = model.loss_function(y_pred, y_train_tensor)

    # backward pass
    loss.backward()

    # parameters update
    with torch.no_grad():  ## to avoid gradient calculation
        model.weights -= learning_rate * model.weights.grad
        model.bias -= learning_rate * model.bias.grad

    # zero gradients
    model.weights.grad.zero_()
    model.bias.grad.zero_()

    # print loss in each epoch
    print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 3.798003450509189
Epoch: 2, Loss: 3.6715953034124285
Epoch: 3, Loss: 3.5376035377466595
Epoch: 4, Loss: 3.398690081156568
Epoch: 5, Loss: 3.254474430437157
Epoch: 6, Loss: 3.101800595789516
Epoch: 7, Loss: 2.9468601479399332
Epoch: 8, Loss: 2.7899555835676613
Epoch: 9, Loss: 2.6314294455815426
Epoch: 10, Loss: 2.4665808786076533
Epoch: 11, Loss: 2.300097203665068
Epoch: 12, Loss: 2.1265345223180705
Epoch: 13, Loss: 1.955506512168512
Epoch: 14, Loss: 1.7831887483730078
Epoch: 15, Loss: 1.6105199755937967
Epoch: 16, Loss: 1.4462545777070785
Epoch: 17, Loss: 1.2963666037378545
Epoch: 18, Loss: 1.1636545043082824
Epoch: 19, Loss: 1.050688296417974
Epoch: 20, Loss: 0.9591048309384429
Epoch: 21, Loss: 0.8887824191278827
Epoch: 22, Loss: 0.8374865725493104
Epoch: 23, Loss: 0.8014396758323324
Epoch: 24, Loss: 0.7765016645793722
Epoch: 25, Loss: 0.7591431689716146


In [34]:
model.weights

tensor([[ 0.4944],
        [-0.0108],
        [-0.1359],
        [-0.1699],
        [-0.3329],
        [ 0.2811],
        [-0.4605],
        [-0.3777],
        [-0.0584],
        [ 0.3104],
        [ 0.2988],
        [-0.0240],
        [ 0.4778],
        [-0.0084],
        [-0.0088],
        [ 0.3882],
        [-0.1475],
        [-0.0856],
        [-0.0159],
        [ 0.3835],
        [-0.3375],
        [-0.0757],
        [ 0.1956],
        [-0.1621],
        [ 0.4365],
        [-0.5502],
        [ 0.1478],
        [ 0.3367],
        [ 0.3885],
        [-0.2961]], dtype=torch.float64, requires_grad=True)

In [35]:
model.bias

tensor([-0.1071], dtype=torch.float64, requires_grad=True)

### Evaluation

In [38]:
# model evaluation
with torch.no_grad():
    y_pred = model.forward(x_test_tensor)
    y_pred = (y_pred > 0.5).float()
    accuracy = (y_pred == y_test_tensor).float().mean()
    print(f'Accuracy: {accuracy.item()}')


Accuracy: 0.5470914244651794
